# Chess Neural Network

Building a neural network that learns to play chess based on my gameplay data.

## Goal
Train a move prediction model to create an AI that plays like me.




In [35]:
# Importing necessary libraries
import pandas as pd
import ast
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

## Step 1: Load Chess Data

Load cleaned data from csv file


In [36]:
df = pd.read_csv("/Users/riteshbhandari/Documents/Dokumentit – Ritesh - MacBook Pro/GitHub/Chess-engine/bot/data-analysis/cleaned_data.csv")
df.head()

,game_id,moves,num_moves,first_move
0,123118274906,"[('white', 'e4', True), ('black', 'e6', False)...",90,e4
1,123118510404,"[('white', 'd4', False), ('black', 'c5', True)...",141,d4
2,123118790014,"[('white', 'e4', False), ('black', 'e5', True)...",46,e4
3,123158939328,"[('white', 'e4', False), ('black', 'd5', True)...",88,e4
4,123160166430,"[('white', 'e4', True), ('black', 'e5', False)...",110,e4


# Step 2: Pre-processing

2.1 Turning features to numerical values (moves.. etc) 

2.2 Encoding the features to be trained

In [37]:
# convert string representation of list to actual list
df["moves"] = df["moves"].apply(ast.literal_eval)

# saving all the moves to single list to be encoded 
every_move = []
for game in df["moves"]:
    for move in game:
        every_move.append(move[1])

every_move[:10]

['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']

In [38]:
# getting all the unique moves
unique_moves = set(every_move)  # just the unique moves
print("Number of different moves:", len(unique_moves))
print()

# give move a number
move_to_number = {}

# turning integer back to moves (for future use)
number_to_move= {}

for i, move in enumerate(unique_moves):
    move_to_number[move] = i
    number_to_move[i] = move

# turning all the numbers into integers
number_moves = []
for move in every_move:
    number_moves.append(move_to_number[move])

# first 10 moves
print("First 10 moves as numbers: ")
print(number_moves[:10])
print()

# first 10 original moves
print("First 10 original moves: ")
print(every_move[:10]) 

Number of different moves: 1927

First 10 moves as numbers: 
[207, 493, 1489, 290, 1640, 627, 1037, 1679, 934, 1536]

First 10 original moves: 
['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']


In [39]:
# Create colors per game
colors_per_game = []

for game in df["moves"]:
    game_colors = []
    for move in game:
        if move[0] == "white":
            game_colors.append(1)
        else:  # black
            game_colors.append(0)
    colors_per_game.append(game_colors)

# Add to DataFrame
df["colors"] = colors_per_game

# Check first row
print(df[["moves", "colors"]].iloc[0])


moves     [(white, e4, True), (black, e6, False), (white...
colors    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [40]:
# turn white or black to integers ( 1  = (White), 0 = (Black) )
teoriat_moves_per_game = []

for game in df["moves"]:
    game_teoriat = []
    for move in game:
        if move[2] == True:
            game_teoriat.append(1)
        else:
            game_teoriat.append(0)
    teoriat_moves_per_game.append(game_teoriat)

# Add to DataFrame
df["teoriat_moves"] = teoriat_moves_per_game

# Check first row
print(df[["moves", "teoriat_moves"]].iloc[0])

moves            [(white, e4, True), (black, e6, False), (white...
teoriat_moves    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [41]:
#combine all the features into a single list of tuples seperated by games 
combined_features_per_game = []

for game_idx in range(len(df)):
    game_combined = []
    moves = df["moves"].iloc[game_idx]
    colors = df["colors"].iloc[game_idx]
    teoriat = df["teoriat_moves"].iloc[game_idx]
    
    for i in range(len(moves)):
        # (color, move_as_integer, your_move)
        game_combined.append((
            colors[i],
            move_to_number[moves[i][1]],  # convert move to integer
            teoriat[i]
        ))
    # Append the combined features for the game
    combined_features_per_game.append(game_combined)

# Add to DataFrame
df["game_data"] = combined_features_per_game

# Check first game data
print(combined_features_per_game[0])

[(1, 207, 1), (0, 493, 0), (1, 1489, 1), (0, 290, 0), (1, 1640, 1), (0, 627, 0), (1, 1037, 1), (0, 1679, 0), (1, 934, 1), (0, 1536, 0), (1, 1073, 1), (0, 46, 0), (1, 421, 1), (0, 1011, 0), (1, 571, 1), (0, 1241, 0), (1, 532, 1), (0, 1502, 0), (1, 1568, 1), (0, 729, 0), (1, 985, 1), (0, 974, 0), (1, 1201, 1), (0, 524, 0), (1, 826, 1), (0, 494, 0), (1, 184, 1), (0, 1438, 0), (1, 57, 1), (0, 1826, 0), (1, 985, 1), (0, 310, 0), (1, 896, 1), (0, 47, 0), (1, 1889, 1), (0, 557, 0), (1, 306, 1), (0, 640, 0), (1, 1251, 1), (0, 1175, 0), (1, 464, 1), (0, 62, 0), (1, 548, 1), (0, 497, 0), (1, 1889, 1), (0, 1027, 0), (1, 478, 1), (0, 372, 0), (1, 49, 1), (0, 1525, 0), (1, 1247, 1), (0, 1762, 0), (1, 788, 1), (0, 1502, 0), (1, 829, 1), (0, 635, 0), (1, 1889, 1), (0, 789, 0), (1, 1251, 1), (0, 904, 0), (1, 972, 1), (0, 61, 0), (1, 16, 1), (0, 1295, 0), (1, 479, 1), (0, 1379, 0), (1, 916, 1), (0, 537, 0), (1, 972, 1), (0, 1379, 0), (1, 916, 1), (0, 537, 0), (1, 1399, 1), (0, 1122, 0), (1, 1650, 1), (

In [42]:
# lets drop the columns with unneccessary data 
df = df.drop(columns=["moves", "colors", "teoriat_moves", "first_move","num_moves"])
df.head()

,game_id,game_data
0,123118274906,"[(1, 207, 1), (0, 493, 0), (1, 1489, 1), (0, 2..."
1,123118510404,"[(1, 1489, 0), (0, 263, 1), (1, 759, 0), (0, 9..."
2,123118790014,"[(1, 207, 0), (0, 934, 1), (1, 1037, 0), (0, 1..."
3,123158939328,"[(1, 207, 0), (0, 759, 1), (1, 934, 0), (0, 30..."
4,123160166430,"[(1, 207, 1), (0, 934, 0), (1, 279, 1), (0, 18..."


### Step 2.2:  Embedded layer Encoding 


In [43]:
unique_moves = set()
for game in df["game_data"]:
    for (color, move, your_move) in game:
        unique_moves.add(move)

print(f"Number of unique moves: {len(unique_moves)}")
print(f"Move range: {min(unique_moves)} to {max(unique_moves)}")

Number of unique moves: 1927
Move range: 0 to 1926
